In [1]:
import pandas as pd
import sympy as sp
import numpy as np
import ast
import spacy
import gensim.downloader as api
from transformers import BertTokenizer, BertModel
import torch
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", message="A parameter name that contains `beta` will be renamed internally to `bias`.")
warnings.filterwarnings("ignore", message="A parameter name that contains `gamma` will be renamed internally to `weight`.")

In [2]:
dfreference= pd.read_csv('Data/refefenceText.csv', header=None)
tokenized_expressionsdf = pd.read_csv('Data/tokenized_expressions.csv', header=None)
dfreference = dfreference.map(lambda x: ast.literal_eval(x))
tokenized_expressionsdf = tokenized_expressionsdf.map(lambda x: ast.literal_eval(x))

In [3]:
def replace_expressions(text, tokenized_expressionsdf):
    def process_value(value):
        if isinstance(value, list):
            # Recursive processing if process_value recibe list type
            processed_list = [process_value(item) for item in value]
            return [item for sublist in processed_list for item in sublist]
        else:
            return [str(value)]

    for key, value in tokenized_expressionsdf.items():
        processed_value = process_value(value)
        text = text.replace(key, ' '.join(processed_value))
    return text

In [4]:
completdf = dfreference.copy()

num_rows, num_cols = dfreference.shape
tokenized_num_rows, tokenized_num_cols = tokenized_expressionsdf.shape

for i in range(num_cols):
    if i >= tokenized_num_cols:
        break
    for j in range(num_rows):
        if j >= tokenized_num_rows:
            break
        row = dfreference.iloc[j, i]
        new_row = []
        for word in row:
            if word.startswith('expression'):
                # Replace the expression with his tokenized version
                parsed = replace_expressions(word, tokenized_expressionsdf.iloc[j, i])
                new_row.append(parsed)
            else:
                new_row.append(word)
        # Update the row in df
        completdf.at[j, dfreference.columns[i]] = new_row


In [5]:
completdf.head()

,0,1,2,3,4,5
0,"[Consider, the, integer, Sum Mul 9 Pow 10 Add ...","[Let, s, express, the, number, in, terms, of, ...","[A, similar, and, simpler, way, to, consider, ...","[We, can, see, that, Equality 9 9, Equality Ad...","[Observe, how, adding, results, in, the, last,...","[Write, N, 9, 99, Add Pow 10 321 -1, {321}, {3..."
1,"[Jenn, randomly, chooses, a, number, J, from, ...","[By, symmetry, the, desired, probability, is, ...","[Ne(B - J, 0), because, Ne(B, J), so, the, pro...","[There, are, Equality 190 190, equally, likely...","[This, problem, is, essentially, asking, how, ...","[Create, a, grid, using, graph, paper, with, 2..."
2,"[In, Mul \triangle P Q R, Equality Mul P R 15,...","[We, know, the, area, of, the, hexagon, ABCDeF...","[Let, R, be, the, origin, Noticing, that, the,...","[Note, that, Mul \triangle P Q R, has, area, 1...","[Knowing, that, Mul \triangle P Q R, has, area...","[Triangle, PQR, is, a, right, triangle, with, ..."
3,"[A, soccer, team, has, 22, available, players,...","[There, are, -3, substitutions, The, number, o...","[There, is, 1, way, of, making, no, substituti...","[We, can, perform, casework, Call, the, substi...","[no, more, solutions, explained]","[no, more, solutions, explained]"
4,"[A, moving, particle, starts, at, the, point, ...","[One, could, recursively, compute, the, probab...","[Obviously, the, only, way, to, reach, (0, 0),...","[Since, the, particle, stops, at, one, of, the...","[All, paths, that, first, hit, the, axes, at, ...","[no, more, solutions, explained]"


In [6]:

def convert_string_to_sentence(lista):
    return ' '.join(lista)  # join the words in a complete sentence
completStringdf = completdf.copy()  

for col in completStringdf.columns:
    if completStringdf[col].dtype == 'object':  # Check the col has texts
        completStringdf[col] = completStringdf[col].apply(convert_string_to_sentence)
completStringdf.head()


,0,1,2,3,4,5
0,Consider the integer Sum Mul 9 Pow 10 Add k -1...,Let s express the number in terms of Pow 10 n ...,A similar and simpler way to consider the init...,We can see that Equality 9 9 Equality Add 9 99...,Observe how adding results in the last term bu...,Write N 9 99 Add Pow 10 321 -1 {321} {321} {32...
1,"Jenn randomly chooses a number J from [1, 2, 3...",By symmetry the desired probability is equal t...,"Ne(B - J, 0) because Ne(B, J) so the probabili...",There are Equality 190 190 equally likely pair...,This problem is essentially asking how many wa...,Create a grid using graph paper with 20 column...
2,In Mul \triangle P Q R Equality Mul P R 15 Equ...,We know the area of the hexagon ABCDeF to be A...,Let R be the origin Noticing that the triangle...,Note that Mul \triangle P Q R has area 150 and...,Knowing that Mul \triangle P Q R has area 150 ...,Triangle PQR is a right triangle with are Equa...
3,A soccer team has 22 available players A fixed...,There are -3 substitutions The number of ways ...,There is 1 way of making no substitutions to t...,We can perform casework Call the substitution ...,no more solutions explained,no more solutions explained
4,"A moving particle starts at the point (4, 4) a...",One could recursively compute the probabilitie...,"Obviously the only way to reach (0, 0) is to g...",Since the particle stops at one of the axes we...,All paths that first hit the axes at the origi...,no more solutions explained


In [7]:
# In order to avoid issues with BERT's handling of `beta` and `gamma`, 
# we are making replacements for these words throughout the DataFrame.
substitutions = {
    'beta': 'betaa',
    'gamma': 'gammaa2'
}

completStringdf.replace(substitutions, regex=True, inplace=True)

In [8]:
# Load tokenizer model and embeddings models
nlp = spacy.load("en_core_web_sm")
glove_model = api.load("glove-wiki-gigaword-300")
word2vec_model = api.load("word2vec-google-news-300")
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [9]:

def tokenize_lemmatize_and_embed(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_punct and not token.is_space]
    
    # GloVe in float32
    glove_embeddings = [np.array(glove_model[token], dtype=np.float32) if token in glove_model else np.zeros(100, dtype=np.float32) for token in tokens]
    
    # Word2Vec in float32
    word2vec_embeddings = [np.array(word2vec_model[token], dtype=np.float32) if token in word2vec_model else np.zeros(300, dtype=np.float32) for token in tokens]
    
    # BERT ein float32
    bert_inputs = bert_tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        bert_outputs = bert_model(**bert_inputs)
    bert_embeddings = bert_outputs.last_hidden_state.mean(dim=1).squeeze().numpy().astype(np.float32)
    
    return tokens, glove_embeddings, word2vec_embeddings, bert_embeddings

In [10]:
tokenizerdf = pd.DataFrame()
for col in completStringdf.columns:
    if completStringdf[col].dtype == 'object':
        tokenizerdf[f'{col}_Tokens'], tokenizerdf[f'{col}_GloVe'], tokenizerdf[f'{col}_Word2Vec'], tokenizerdf[f'{col}_BERT'] = zip(*completStringdf[col].apply(lambda x: tokenize_lemmatize_and_embed(' '.join(x)) if isinstance(x, list) else tokenize_lemmatize_and_embed(x)))

In [11]:
tokenizerdf.head()

,0_Tokens,0_GloVe,0_Word2Vec,0_BERT,1_Tokens,1_GloVe,1_Word2Vec,1_BERT,2_Tokens,2_GloVe,...,3_Word2Vec,3_BERT,4_Tokens,4_GloVe,4_Word2Vec,4_BERT,5_Tokens,5_GloVe,5_Word2Vec,5_BERT
0,"[consider, the, integer, Sum, Mul, 9, Pow, 10,...","[[0.38731, 0.053821, 0.29723, 0.048756, 0.1740...","[[-0.06347656, 0.09082031, 0.1953125, 0.093261...","[-0.09016116, -0.11996196, 0.46167573, -0.2819...","[let, s, express, the, number, in, term, of, P...","[[0.31646, -0.0018851, -0.36329, -0.11607, -0....","[[0.20800781, 0.039794922, 0.25, 0.24414062, -...","[-0.18826374, 0.0021300127, 0.3243757, -0.1697...","[a, similar, and, simple, way, to, consider, t...","[[-0.29712, 0.094049, -0.096662, -0.344, -0.18...",...,"[[-0.017944336, 0.1171875, 0.052734375, 0.2558...","[-0.037111796, -0.017886223, 0.48509687, -0.19...","[observe, how, add, result, in, the, last, ter...","[[0.25236, 0.17171, -0.27513, -0.22323, -0.644...","[[-0.11035156, -0.013916016, 0.11376953, 0.177...","[-0.26127538, -0.100837104, 0.35027117, -0.301...","[write, N, 9, 99, Add, Pow, 10, 321, -1, 321, ...","[[-0.32122, 0.024329, -0.49585, -0.41928, -0.1...","[[-0.03930664, -0.18164062, 0.037597656, -0.04...","[-0.00815094, 0.08880436, 0.4837909, -0.340913..."
1,"[Jenn, randomly, choose, a, number, j, from, 1...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-0.025512695, -0.042236328, -0.23632812, -0....","[-0.31445926, 0.4070755, 0.46623844, -0.298210...","[by, symmetry, the, desire, probability, be, e...","[[0.073515, 0.25736, 0.2359, 0.19299, -0.29611...","[[-0.115722656, -0.03149414, 0.15917969, 0.138...","[-0.19177645, -0.15448825, 0.47980326, -0.2328...","[Ne(B, J, 0, because, Ne(B, J, so, the, probab...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",...,"[[0.09423828, -0.022827148, 0.052246094, 0.026...","[-0.05002716, 0.16393934, 0.7842918, -0.159680...","[this, problem, be, essentially, ask, how, man...","[[-0.20437, 0.16431, 0.041794, -0.13708, -0.29...","[[0.109375, 0.140625, -0.03173828, 0.16601562,...","[-0.104582764, 0.038583174, 0.5119848, -0.1771...","[create, a, grid, use, graph, paper, with, 20,...","[[-0.36201, 0.26625, -0.498, -0.29232, 0.01494...","[[-0.03857422, 0.020629883, -0.03149414, -0.05...","[-0.32661444, -0.11376465, 0.4146434, -0.28907..."
2,"[in, Mul, \triangle, p, Q, r, Equality, Mul, p...","[[-0.44399, 0.12817, -0.25247, -0.18582, -0.16...","[[0.0703125, 0.08691406, 0.087890625, 0.0625, ...","[-0.44343543, 0.15149468, 0.5914894, -0.135701...","[we, know, the, area, of, the, hexagon, ABCDeF...","[[-0.050835, 0.24826, -0.19384, 0.083809, -0.1...","[[-0.017944336, 0.1171875, 0.052734375, 0.2558...","[-0.13557069, 0.16244687, 0.48237172, -0.35520...","[let, r, be, the, origin, Noticing, that, the,...","[[0.31646, -0.0018851, -0.36329, -0.11607, -0....",...,"[[-0.087402344, 0.25390625, -0.21777344, -0.13...","[-0.28536543, -0.047120534, 0.54667264, -0.256...","[know, that, Mul, \triangle, p, Q, R, have, ar...","[[-0.21054, 0.1382, 0.035328, 0.03977, -0.1091...","[[-0.05493164, -0.1171875, 0.027832031, 0.0747...","[-0.22818379, 0.019595118, 0.5663583, -0.20902...","[Triangle, PQR, be, a, right, triangle, with, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-0.17285156, -0.28320312, -0.07421875, 0.216...","[-0.33113176, 0.087249026, 0.246714, -0.215530..."
3,"[a, soccer, team, have, 22, available, player,...","[[-0.29712, 0.094049, -0.096662, -0.344, -0.18...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[-0.50705576, -0.090016216, 0.150927, -0.13018...","[there, be, -3, substitution, the, number, of,...","[[-0.47618, 0.069478, -0.013086, 0.1742, -0.31...","[[0.09423828, -0.022827148, 0.052246094, 0.026...","[-0.19937521, -0.046121426, 0.42295498, -0.110...","[there, be, 1, way, of, make, no, substitution...","[[-0.47618, 0.069478, -0.013086, 0.1742, -0.31...",...,"[[-0.017944336, 0.1171875, 0.052734375, 0.2558...","[-0.49748585, -0.02424635, 0.16674419, -0.1833...","[no, more, solution, explain]","[[-0.16843, -0.0376

In [12]:
#tokenizerdf.to_csv('Data/tokenizer.csv', index= False)

# Note: CSV format does not perform well with vectors. Therefore, we will check for file types that support float32 data 
# and convert them to JSON, which preserves vectors better, especially for models like GloVe, Word2Vec, and BERT.

In [13]:
for col in tokenizerdf.columns:
    if col.endswith('_GloVe') or col.endswith('_Word2Vec') or col.endswith('_BERT'):
        print(f"Columna: {col}, Tipo de datos: {type(tokenizerdf[col].iloc[0])}, dtype: {tokenizerdf[col].dtype}")
        for i, embedding in enumerate(tokenizerdf[col]):
            if isinstance(embedding, list):
                print(f"  Element {i}: Data type: {type(embedding[0])}, dtype: {np.array(embedding).dtype}")
            elif isinstance(embedding, np.ndarray):
                print(f"  Element {i}: Data type: {type(embedding)}, dtype: {embedding.dtype}")

Columna: 0_GloVe, Tipo de datos: <class 'list'>, dtype: object
  Element 0: Data type: <class 'numpy.ndarray'>, dtype: object
  Element 1: Data type: <class 'numpy.ndarray'>, dtype: object
  Element 2: Data type: <class 'numpy.ndarray'>, dtype: object
  Element 3: Data type: <class 'numpy.ndarray'>, dtype: float32
  Element 4: Data type: <class 'numpy.ndarray'>, dtype: object
  Element 5: Data type: <class 'numpy.ndarray'>, dtype: object
  Element 6: Data type: <class 'numpy.ndarray'>, dtype: object
  Element 7: Data type: <class 'numpy.ndarray'>, dtype: object
  Element 8: Data type: <class 'numpy.ndarray'>, dtype: object
  Element 9: Data type: <class 'numpy.ndarray'>, dtype: object
  Element 10: Data type: <class 'numpy.ndarray'>, dtype: object
  Element 11: Data type: <class 'numpy.ndarray'>, dtype: object
  Element 12: Data type: <class 'numpy.ndarray'>, dtype: object
  Element 13: Data type: <class 'numpy.ndarray'>, dtype: object
  Element 14: Data type: <class 'numpy.ndarray'>, 

In [14]:

# Save DataFrame in JSON format fot the next Section
tokenizerdf.to_json('Data/jsonFile_withTokens.json', orient='records', lines=True, default_handler=lambda x: x.tolist() if isinstance(x, np.ndarray) else x)